# THIS NOTEBOOK CAN BE USED TO COMPARE WVFs FROM ANY SOURCE
### 1. Load the WVFs from the source. Note the following variables to load the data:
    - pretrigger: the number of samples before the peak. Wvfs are automatically aligned to the peak.
    - sampling: the sampling rate of the data.
    - convert_sampling: None or a number. If a number, the data will be resampled to this sampling rate.
    - same_length: True or False. If True, all WVFs will be truncated or padded to the wvf_length. If False, the WVFs will be plotted as is.
    - wvf_length: desired length of the wvf. If the wvf is shorter than this, it will be padded with zeros. If it is longer, it will be truncated.

In [1]:
import sys; sys.path.insert(0, '../'); from lib.__init__ import *

pretrigger = 100
wvf_length = 2500

ir02_wvfs = import_templates(template_path="wvfs/IR02/",these_types=["LASER","SPE"],pretrigger=pretrigger,sampling=4e-9,convert_sampling=None,same_length=True,wvf_length=wvf_length,debug=False)
larsoft_wvfs = import_templates(template_path="wvfs/IR02/",these_types=["TEMPLATE"],pretrigger=pretrigger,sampling=16e-9,convert_sampling=True,new_sampling=4e-9,same_length=True,wvf_length=wvf_length,debug=False)
milano_wvfs = import_templates(template_path="wvfs/MILANO/",these_types=["TEMPLATE"],pretrigger=pretrigger,sampling=16e-9,convert_sampling=True,new_sampling=4e-9,same_length=True,wvf_length=wvf_length,debug=False)

wvfs = ir02_wvfs + larsoft_wvfs + milano_wvfs

Done!
Done!
Done!


### 2. Combine the WVFs into a single dataframe.

In [2]:
df = pd.DataFrame(wvfs)
# display(df)

### 3. Plot the WVFs.

In [3]:
# Combine all templates into a single plot.
fig = px.line(df.explode(["ADC","TIME"]),
                x="TIME",
                y="ADC",
                color="OV",
                line_dash="MODEL",
                facet_col="INST",
                facet_row="TYPE",
                title="Templates")

fig = format_coustom_plotly(fig,figsize=(None,600))
fig.update_yaxes(matches=None)
fig.show()

In [4]:
# Make an animation of succesive wvfs.
fig = px.line(df[(df["INST"] == "IR02") & (df["TYPE"] == "LASER") & (df["MODEL"] == "HPK")].explode(["ADC","TIME"]),
                x="TIME",
                y="ADC",
                color="OV",
                category_orders={"OV": [3,2,1]},
                animation_frame="OV")
fig = format_coustom_plotly(fig,figsize=(None,600))
fig.update_layout(showlegend=False)
fig.show()

In [14]:
# Normalize the wvfs to have a better comparison.
norm_df = normalize_wvfs(df)
fig = px.line(norm_df[(norm_df["TYPE"] == "TEMPLATE") & (norm_df["OV"] != 1) & (norm_df["OV"] != 3)].explode(["ADC","TIME"]),
                x="TIME",
                y="ADC",
                color="INST",
                facet_col="MODEL",
)
fig = format_coustom_plotly(fig,figsize=(None,600),tickformat=(",.2s",",.2f"))
fig.update_yaxes(matches=None)
fig.show()



### 4. Generate template wvfs from "LASER" and "SPE" templates.

In [6]:
# This function can be found in lib/io.py and is used to generate the templates from "LASER" and "SPE" types inside the df dataframe.

larsoft_wvfs = generate_templates(df,sampling=4e-9,
                                new_sampling=16e-9,
                                ovs=[1,2,3],
                                models=["FBK","HPK"],
                                type_folder="TEST",
                                main_folder="IR02",
                                max_length=4000,
                                save=False,
                                debug=False)

### 4. Enjoy!